# Porque não usar crc32 como surrogate key em datasets grandes

In [11]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Ex9") \
    .getOrCreate()

22/07/05 14:34:22 WARN Utils: Your hostname, computador resolves to a loopback address: 127.0.1.1; using 10.0.0.135 instead (on interface wlp2s0)
22/07/05 14:34:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/05 14:34:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
df = spark.read.csv('./data/sales06-12-2020-10-20-33.csv', header=True)

In [6]:
df.show()

+-------+-------+--------+----------+--------+-------------+-----------+
|PROD_ID|CUST_ID| TIME_ID|CHANNEL_ID|PROMO_ID|QUANTITY_SOLD|AMOUNT_SOLD|
+-------+-------+--------+----------+--------+-------------+-----------+
|     13|    987|10-01-98|         3|     999|            1|    1232.16|
|     13|   1660|10-01-98|         3|     999|            1|    1232.16|
|     13|   1762|10-01-98|         3|     999|            1|    1232.16|
|     13|   1843|10-01-98|         3|     999|            1|    1232.16|
|     13|   1948|10-01-98|         3|     999|            1|    1232.16|
|     13|   2273|10-01-98|         3|     999|            1|    1232.16|
|     13|   2380|10-01-98|         3|     999|            1|    1232.16|
|     13|   2683|10-01-98|         3|     999|            1|    1232.16|
|     13|   2865|10-01-98|         3|     999|            1|    1232.16|
|     13|   4663|10-01-98|         3|     999|            1|    1232.16|
|     13|   5203|10-01-98|         3|     999|     

## Criação do id a partir de dados das colunas

In [7]:
df = df.withColumn('id', concat(col('PROD_ID'), col('CUST_ID'), col('TIME_ID'), col('CHANNEL_ID')))

In [8]:
df.show()

+-------+-------+--------+----------+--------+-------------+-----------+----------------+
|PROD_ID|CUST_ID| TIME_ID|CHANNEL_ID|PROMO_ID|QUANTITY_SOLD|AMOUNT_SOLD|              id|
+-------+-------+--------+----------+--------+-------------+-----------+----------------+
|     13|    987|10-01-98|         3|     999|            1|    1232.16|  1398710-01-983|
|     13|   1660|10-01-98|         3|     999|            1|    1232.16| 13166010-01-983|
|     13|   1762|10-01-98|         3|     999|            1|    1232.16| 13176210-01-983|
|     13|   1843|10-01-98|         3|     999|            1|    1232.16| 13184310-01-983|
|     13|   1948|10-01-98|         3|     999|            1|    1232.16| 13194810-01-983|
|     13|   2273|10-01-98|         3|     999|            1|    1232.16| 13227310-01-983|
|     13|   2380|10-01-98|         3|     999|            1|    1232.16| 13238010-01-983|
|     13|   2683|10-01-98|         3|     999|            1|    1232.16| 13268310-01-983|
|     13| 

## Criação da crc32_key a partir do id

In [9]:
df = df.withColumn('crc32_key', crc32(col('id')))

In [10]:
df.show()

+-------+-------+--------+----------+--------+-------------+-----------+----------------+----------+
|PROD_ID|CUST_ID| TIME_ID|CHANNEL_ID|PROMO_ID|QUANTITY_SOLD|AMOUNT_SOLD|              id| crc32_key|
+-------+-------+--------+----------+--------+-------------+-----------+----------------+----------+
|     13|    987|10-01-98|         3|     999|            1|    1232.16|  1398710-01-983|2544656211|
|     13|   1660|10-01-98|         3|     999|            1|    1232.16| 13166010-01-983|2811527586|
|     13|   1762|10-01-98|         3|     999|            1|    1232.16| 13176210-01-983| 952376816|
|     13|   1843|10-01-98|         3|     999|            1|    1232.16| 13184310-01-983|4261302398|
|     13|   1948|10-01-98|         3|     999|            1|    1232.16| 13194810-01-983|2663114726|
|     13|   2273|10-01-98|         3|     999|            1|    1232.16| 13227310-01-983| 870429392|
|     13|   2380|10-01-98|         3|     999|            1|    1232.16| 13238010-01-983|34

## Criação da coluna de dados duplicados, considerando o número de crc32_key únicos

In [39]:
df = df.withColumn('duplicates', row_number().over(Window.partitionBy('crc32_key').orderBy('crc32_key')))

In [40]:
df.show()

+-------+-------+--------+----------+--------+-------------+-----------+-----------------+---------+----------+
|PROD_ID|CUST_ID| TIME_ID|CHANNEL_ID|PROMO_ID|QUANTITY_SOLD|AMOUNT_SOLD|               id|crc32_key|duplicates|
+-------+-------+--------+----------+--------+-------------+-----------+-----------------+---------+----------+
|     30|   1578|30-07-98|         2|     999|            1|      10.79|  30157830-07-982|   374172|         1|
|     24|   7080|19-04-99|         3|     999|            1|      51.51|  24708019-04-993|  4109626|         1|
|    115|   5034|24-04-99|         3|     999|            1|      10.95| 115503424-04-993|  4856212|         1|
|     41|   2352|04-09-99|         2|     999|            1|      46.35|  41235204-09-992|  6162121|         1|
|    119|   2682|18-09-99|         3|     999|            1|       8.37| 119268218-09-993|  8139065|         1|
|     30|  10675|12-03-01|         3|     999|            1|      10.36| 301067512-03-013|  8273566|    

## Filtragem para o número de crc32_key duplicados (maior que um)

In [19]:
df.filter('duplicates >1').show()

+-------+-------+--------+----------+--------+-------------+-----------+-----------------+----------+----------+
|PROD_ID|CUST_ID| TIME_ID|CHANNEL_ID|PROMO_ID|QUANTITY_SOLD|AMOUNT_SOLD|               id| crc32_key|duplicates|
+-------+-------+--------+----------+--------+-------------+-----------+-----------------+----------+----------+
|     22|   2171|25-06-01|         3|     999|            1|      25.79|  22217125-06-013|3517338981|         2|
|    117|   4302|27-03-01|         3|     999|            1|       9.75| 117430227-03-013| 611822206|         2|
|    115|   4302|27-03-01|         2|     999|            1|       9.41| 115430227-03-012|2988334661|         2|
|    116|  10715|06-07-01|         3|     999|            1|      12.85|1161071506-07-013| 730148726|         2|
|     30|   7784|03-09-00|         2|     999|            1|       9.35|  30778403-09-002| 170251350|         2|
|    116|  10715|06-07-01|         2|     999|            1|      12.86|1161071506-07-012|155203

## Criação de 

In [21]:
df.createOrReplaceTempView('sales_v')

In [37]:
query = 'SELECT * FROM sales_v WHERE crc32_key in (SELECT crc32_key FROM sales_v WHERE duplicates > 1)'

In [38]:
spark.sql(query).show()

+-------+-------+--------+----------+--------+-------------+-----------+-----------------+----------+----------+
|PROD_ID|CUST_ID| TIME_ID|CHANNEL_ID|PROMO_ID|QUANTITY_SOLD|AMOUNT_SOLD|               id| crc32_key|duplicates|
+-------+-------+--------+----------+--------+-------------+-----------+-----------------+----------+----------+
|    123|   4415|18-02-01|         2|     999|            1|      57.86| 123441518-02-012|3517338981|         1|
|     22|   2171|25-06-01|         3|     999|            1|      25.79|  22217125-06-013|3517338981|         2|
|     24|   3336|14-02-00|         2|     999|            1|      48.33|  24333614-02-002| 611822206|         1|
|    117|   4302|27-03-01|         3|     999|            1|       9.75| 117430227-03-013| 611822206|         2|
|     26|   3336|14-02-00|         3|     999|            1|     162.62|  26333614-02-003|2988334661|         1|
|    115|   4302|27-03-01|         2|     999|            1|       9.41| 115430227-03-012|298833